## Notebook Magic

In [ ]:
%matplotlib inline
%load_ext autoreload

## Imports

In [ ]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

## Get Pedidos Table

In [ ]:
df = pd.read_parquet("s3://iefp-unemployment/intermediate/clean/pedidos.parquet")

In [ ]:
df = df.sort_values(["ute_id", "data_movimento"])

In [ ]:
end_count = len(df[df["tipo_movimento"].isin([21, 31])])
start_count = len(df[df["tipo_movimento"] == 11])
print(start_count)
print(end_count)

In [ ]:
df_dem = df.copy()
df_dem = df_dem[df_dem["tipo_movimento"] == 11]
df_dem["data_movimento"] = df_dem["data_movimento"].dt.date
df_dem = df_dem.groupby(["ute_id", "data_movimento"]).first()
print(df_dem.shape)
df_dem = df_dem.drop_duplicates()
print(df_dem.shape)
df_dem.describe(include='all')

In [ ]:
df_t = df.loc[df["tipo_movimento"].isin(["11", "21", "31"]), ["ute_id", "data_movimento", "tipo_movimento", "motivo_inscricao", "motivo_anulacao"]].copy()
df_t["journey_start"] = df_t["tipo_movimento"] == 11
df_t['journey_count'] = df_t.groupby("ute_id")["journey_start"].cumsum()
df_t["tipo_movimento"] = df_t["tipo_movimento"].astype(np.int64).astype('str')

# Note: What about multiple registers (21) at the same day?
# Note: What about cases where multiple exits (31, 21) after each other?
# Note: What about cases where exit is successful vocational training? In newer times these people just go to 'busy'
df_t = df_t.pivot_table(index = ['ute_id', 'journey_count'], columns = 'tipo_movimento', aggfunc=np.min)
df_t.columns = ['_'.join(col).strip() for col in df_t.columns.values]
df_t = df_t.reset_index()

In [ ]:
cols = ['ute_id',
 'journey_count',
 'data_movimento_11',
 'data_movimento_21',
 'data_movimento_31',
 'motivo_anulacao_31',
 'motivo_inscricao_11',
]
t_cols = ['user_id', 'journey_count', 'register_date', 'exit_date_21', 'exit_date_31', 'exit_reason', 'register_reason']
df_t = df_t[cols]
df_t.columns = t_cols

In [ ]:
print(df_t.shape)
df_t = df_t[df_t["journey_count"] > 0]
df_t = df_t[(df_t["exit_date_21"].notna()) | (df_t["exit_date_31"].notna())]
print(df_t.shape)